<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao34_RAG_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Colab: Exemplo de RAG (Retrieval-Augmented Generation)

Este notebook demonstra um fluxo RAG simples do zero.
 1. Instala bibliotecas.
 2. Configura a API Key (necessário).
 3. Define uma "base de conhecimento" (corpus).
 4. Cria embeddings para o corpus.
 5. Indexa os embeddings com FAISS (Retrieval).
 6. Configura o LLM (Gemini) (Generation).
 7. Executa uma consulta RAG (Retrieve -> Augment -> Generate).

In [1]:
!pip install -q -U google-generativeai sentence-transformers faiss-cpu

import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import textwrap
from google.colab import userdata

print("Bibliotecas instaladas com sucesso!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 26.7 MB/s eta 0:00:00
Bibliotecas instaladas com sucesso!


In [2]:
# ---
# ⚠️ **Ação Necessária** ⚠️
# 1. No menu à esquerda do Colab, clique no ícone de chave (Secrets).
# 2. Adicione um novo "secret" com o nome "GOOGLE_API_KEY".
# 3. Cole sua API Key do Google AI Studio nesse "secret".
#
# Você pode obter uma API Key em: https://aistudio.google.com/app/apikey
# ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada com sucesso!")
except userdata.SecretNotFoundError:
    print("Erro: 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab.")
    print("Por favor, adicione sua API Key do Google AI Studio aos Secrets (ícone de chave à esquerda) com o nome 'GOOGLE_API_KEY'.")
except Exception as e:
    print(f"Ocorreu um erro ao configurar a API: {e}")

API Key configurada com sucesso!


In [3]:
# Esta é a nossa pequena "base de dados" de onde o RAG vai "puxar" informações.
corpus = [
    "O sol é uma estrela localizada no centro do Sistema Solar.",
    "A fotossíntese é o processo pelo qual as plantas usam a luz solar para produzir alimentos.",
    "O oceano Atlântico é o segundo maior oceano do mundo.",
    "A Torre Eiffel, localizada em Paris, foi concluída em 1889.",
    "A Amazônia é a maior floresta tropical do mundo e abriga milhões de espécies.",
    "O ciclo da água descreve o movimento contínuo da água na Terra, através da evaporação, condensação e precipitação."
]

print(f"Corpus de conhecimento com {len(corpus)} documentos.")

Corpus de conhecimento com 6 documentos.


In [4]:
# Usamos 'sentence-transformers' para converter nossos textos em vetores numéricos.
# 'paraphrase-multilingual-MiniLM-L12-v2' é um bom modelo multilíngue.
print("Carregando modelo de embeddings...")
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("Modelo carregado.")

Carregando modelo de embeddings...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo carregado.


In [5]:
# 1. Gerar embeddings para todo o nosso corpus
print("Gerando embeddings para o corpus...")
corpus_embeddings = embedding_model.encode(corpus)

# 2. Definir a dimensão dos vetores
d = corpus_embeddings.shape[1]

# 3. Criar um índice FAISS (IndexFlatL2 é um índice simples de busca por similaridade)
print("Criando índice FAISS...")
index = faiss.IndexFlatL2(d)

# 4. Adicionar os vetores ao índice
index.add(corpus_embeddings)
print(f"Índice FAISS criado e {index.ntotal} vetores adicionados.")

Gerando embeddings para o corpus...
Criando índice FAISS...
Índice FAISS criado e 6 vetores adicionados.


In [9]:
# Usaremos o Gemini (neste caso, o 1.5-flash) para gerar a resposta final.
llm = genai.GenerativeModel(
  model_name='gemini-2.5-flash',
  system_instruction="Você é um assistente prestativo. Responda à pergunta do usuário com base **apenas** no contexto fornecido."
)

In [10]:
def executar_consulta_rag(pergunta, k=2):
    """
    Executa o fluxo RAG completo:
    1. Retrieve: Busca os 'k' documentos mais relevantes.
    2. Augment: Monta o prompt com o contexto e a pergunta.
    3. Generate: Gera a resposta usando o LLM.
    """
    print(f"--- Nova Consulta RAG ---")
    print(f"Pergunta original: {pergunta}\n")

    # --- 1. RETRIEVAL ---
    # Converter a pergunta em um vetor (embedding)
    pergunta_embedding = embedding_model.encode([pergunta])

    # Buscar os k vizinhos mais próximos no índice FAISS
    # D = Distâncias, I = Índices dos documentos no corpus
    D, I = index.search(pergunta_embedding, k)

    # Obter os textos dos documentos encontrados
    documentos_relevantes = [corpus[i] for i in I[0]]

    print(f"Documentos recuperados (Contexto):")
    for i, doc in enumerate(documentos_relevantes):
        print(f"  {i+1}. {doc}")
    print("\n")

    # --- 2. AUGMENTATION ---
    # Criar o prompt aumentado, injetando o contexto
    contexto = "\n".join(documentos_relevantes)
    prompt_aumentado = f"""
    **Contexto:**
    {contexto}

    **Pergunta:**
    {pergunta}

    **Resposta:**
    """

    # --- 3. GENERATION ---
    print("Gerando resposta com base no contexto...")
    # Gerar a resposta usando o LLM (Gemini)
    try:
        resposta = llm.generate_content(prompt_aumentado)
        return textwrap.fill(resposta.text, width=80)
    except Exception as e:
        return f"Erro ao gerar resposta: {e}"

In [11]:
# --- Teste 1: Pergunta cuja resposta ESTÁ no corpus ---
pergunta1 = "O que é fotossíntese?"
resposta1 = executar_consulta_rag(pergunta1, k=1)

print("\n--- RESPOSTA FINAL (RAG) ---")
print(resposta1)
print("-" * 30)

# --- Teste 2: Pergunta cuja resposta ESTÁ no corpus, mas é mais específica ---
pergunta2 = "Onde fica a Torre Eiffel e quando foi concluída?"
resposta2 = executar_consulta_rag(pergunta2, k=2) # Pedimos 2 docs para garantir

print("\n--- RESPOSTA FINAL (RAG) ---")
print(resposta2)
print("-" * 30)

# --- Teste 3: Pergunta cuja resposta NÃO ESTÁ no corpus ---
# O LLM (devido ao system_instruction) deve indicar que não sabe.
pergunta3 = "Quem foi o primeiro presidente do Brasil?"
resposta3 = executar_consulta_rag(pergunta3, k=2)

print("\n--- RESPOSTA FINAL (RAG) ---")
print(resposta3)
print("-" * 30)

--- Nova Consulta RAG ---
Pergunta original: O que é fotossíntese?

Documentos recuperados (Contexto):
  1. A fotossíntese é o processo pelo qual as plantas usam a luz solar para produzir alimentos.


Gerando resposta com base no contexto...

--- RESPOSTA FINAL (RAG) ---
A fotossíntese é o processo pelo qual as plantas usam a luz solar para produzir
alimentos.
------------------------------
--- Nova Consulta RAG ---
Pergunta original: Onde fica a Torre Eiffel e quando foi concluída?

Documentos recuperados (Contexto):
  1. A Torre Eiffel, localizada em Paris, foi concluída em 1889.
  2. O ciclo da água descreve o movimento contínuo da água na Terra, através da evaporação, condensação e precipitação.


Gerando resposta com base no contexto...

--- RESPOSTA FINAL (RAG) ---
A Torre Eiffel fica em Paris e foi concluída em 1889.
------------------------------
--- Nova Consulta RAG ---
Pergunta original: Quem foi o primeiro presidente do Brasil?

Documentos recuperados (Contexto):
  1. A Ama